In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2022-06-02 20:10:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  69.7MB/s    in 0.2s    

2022-06-02 20:10:36 (69.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]



In [18]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [20]:
get_ipython().system_raw('./ngrok authtoken 265lteDvFQPBekuZDZLqxXweq5s_5RbJoqbkpRU3BvmKrTJCS')

In [21]:
get_ipython().system_raw('./ngrok http 8501 &')

In [22]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: http://8940-34-86-182-179.ngrok.io


In [23]:
%%writefile nyc.py

import pandas as pd
import numpy as np
import streamlit as st
import codecs
import re

DATA_URL = '/content/drive/MyDrive/citibike-tripdata.csv'
COLUMNA = 'started_at'

st.title('Cicle Rides in NYC')
sidebar = st.sidebar

@st.cache
def cargar_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows, encoding_errors='ignore')
    lowercase = lambda x: str(x).lower()
    data.rename(lowercase, axis='columns', inplace=True)
    data[COLUMNA] = pd.to_datetime(data[COLUMNA])
    return data

checkbox_data = sidebar.checkbox("Mostrar todos los datos")
if checkbox_data:
    estado = st.text('Cargando...')
    data = cargar_data(500)    
    estado.text("¡Cargado! (usando st.cache)")
    st.subheader('Todos los datos')
    st.dataframe(data)

checkbox_hora = sidebar.checkbox("Mostrar los datos por hora")
if checkbox_hora:
    estado = st.text('Cargando...')
    data = cargar_data(500)
    estado.text("¡Cargado! (usando st.cache)")
    st.subheader('Numero de recorridos por hora')
    values = np.histogram(data[COLUMNA].dt.hour, bins=24, range=(0,24))[0]
    st.bar_chart(values)


filtro_hora = sidebar.slider('HORA', 0, 23, 17)
if filtro_hora:
    estado = st.text('Cargando...')
    data = cargar_data(500)
    estado.text("¡Cargado! (usando st.cache)")
    data_rename = data.rename(columns = {'start_lat': 'lat', 'start_lng': 'lon'}, inplace = False)
    filtrar_data = data_rename[data_rename[COLUMNA].dt.hour == filtro_hora]
    st.subheader('Mapa de los recorridos iniciados a las %s:00' % filtro_hora)
    st.map(filtrar_data)

Overwriting nyc.py


In [24]:
!streamlit run /content/nyc.py

2022-06-02 20:11:16.489 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.86.182.179:8501

  Stopping...
  Stopping...
